# Hướng dẫn hoàn chỉnh về PySpark và SparkSQL.

In [1]:
# chúng tôi sử dụng thư viện findspark để xác định vị trí spark trên máy cục bộ của chúng
import findspark
findspark.init('C:/spark/spark/spark-3.2.3-bin-hadoop2.7')

In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 1. Khởi tạo phiên Spark

In [3]:
sc = SparkSession.builder.appName("PysparkExample").config ("spark.sql.shuffle.partitions","50").config("spark.driver.maxResultSize","5g").config("spark.sql.execution.arrow.enabled", "true").getOrCreate()
print(sc)

# SparkContext

<p><a href="http://172.16.2.39:4040">Spark UI</a></p>
<dl>
<dt>Version</dt>
<dd><code>v3.2.3</code></dd>
<dt>Master</dt>
<dd><code>local[*]</code></dd>
<dt>AppName</dt>
<dd><code>PysparkExample</code></dd>
</dl>
</div>

# 2. Tải dữ liệu

In [4]:
#JSON
dataframe = sc.read.json('C:/data/nyt2.json')
# dataframe_txt = sc.read.text('./data/text_data.txt')
#CSV files
# dataframe_csv = sc.read.csv('./data/csv_data.csv')
#PARQUET files
# dataframe_parquet = sc.read.load('./data/parquet_data.parquet')

# Xemdữ liệu với show()

In [25]:
dataframe.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|           

# Kiểmtra đơn giản dữ liệu.

In [5]:
# Trả về tên cột và kiểu dữ liệu của dataframe
dataframe.dtypes
# Hiển thị nội dung của dataframe
dataframe.show()
# Trả về n hàng đầu tiên
dataframe.head()
# Trả về hàng đầu tiên
dataframe.first()
# Return first n rows
dataframe.take(5)
# Tính toán số liệu thống kê tóm tắt
dataframe.describe().show()
# Trả về các cột của dataframe
dataframe.columns
# Đếm số hàng trong khung dataframe
dataframe.count()
# Đếm số hàng riêng biệt trong khung dữ liệu
dataframe.distinct().count()
# In các kế hoạch bao gồm vật lý và logic
dataframe.explain(4)

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|       Little, Br

TypeError: extended (optional) and mode (optional) should be a string and bool; however, got [<class 'int'>].

## 3. Các hàm phổ biến hữu ích

## [1] Xóa các giá trị trùng lặp

In [6]:
dataframe_dropdup = dataframe.dropDuplicates()
dataframe_dropdup.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+----------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|       publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+----------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|Clive Cussler wit...|{{1213401600000}}|Juan Cabrillo and...|{26.95, null}|{{1214697600000}}|          Putnam| {4}|           {3}|         PLAGUE SHIP|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|      Jeffery Deaver|{{1215820800000}}|Detectives Lincol...|    {null, 0}|{{1217116800000}}|Simon & Schuster|{20}|           

## [2] Phép toán 'Select'

In [7]:
#Hiển thị tất cả các mục trong cột tiêu đề
dataframe.select("author").show(10)
#Hiển thị các cột title, author, rank, price
dataframe.select("author", "title", "rank", "price").show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows

+--------------------+--------------------+----+-------------+
|              author|               title|rank|        price|
+--------------------+--------------------+----+-------------+
|       Dean R Koontz|           ODD HOURS| {1}|   {null, 27}|
|     Stephenie Meyer|            THE HOST| {2}|{25.99, null}|
|        Emily Giffin|LOVE THE ONE YOU'...| {3}|{24.95, null}|
|   Patricia Cornwell|           THE FRONT| {4}|{22.95, null}|
|     Chuck Palahniuk|               SNUFF| {5}|{24.95, null}|
|James Patterson a...|SUNDAYS AT TIFFANY’S| {6}|{24.99, null}|
|       John Sandford|        PHANTOM PREY| {7}|{26.95, null}|
|       Jimmy Buffett|

## [3] Phép toán 'When'

In [8]:
# Hiển thị tiêu đề và gán 0 hoặc 1 tùy thuộc vào tiêu đề
dataframe.select("title", when(dataframe.title != 'ODD HOURS', 1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

## [4] Phép toán 'isin'

In [9]:
# Hiển thị các hàng có tác giả được chỉ định nếu trong các tùy chọn đã cho
dataframe [dataframe.author.isin("John Sandford", "Emily Giffin")].show(5)

+--------------------+--------------------+-------------+-----------------+--------------------+-------------+-----------------+------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|        price|   published_date|   publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+-------------+-----------------+------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...| Emily Giffin|{{1211587200000}}|A woman's happy m...|{24.95, null}|{{1212883200000}}|St. Martin's| {3}|           {2}|LOVE THE ONE YOU'...|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|John Sandford|{{1211587200000}}|The Minneapolis d...|{26.95, null}|{{1212883200000}}|      Putnam| {7}|           {4}|        PHANTOM PREY|          {3}|
|{5b4aa4ead3089

## [5] Phép toán 'Like'

In [10]:
# Hiển thị tác giả và tiêu đề là TRUE nếu tiêu đề có từ " THE " trong tiêu đề
dataframe.select("author", "title", dataframe.title.like("% THE %")).show(15)

+--------------------+--------------------+------------------+
|              author|               title|title LIKE % THE %|
+--------------------+--------------------+------------------+
|       Dean R Koontz|           ODD HOURS|             false|
|     Stephenie Meyer|            THE HOST|             false|
|        Emily Giffin|LOVE THE ONE YOU'...|              true|
|   Patricia Cornwell|           THE FRONT|             false|
|     Chuck Palahniuk|               SNUFF|             false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|             false|
|       John Sandford|        PHANTOM PREY|             false|
|       Jimmy Buffett|          SWINE NOT?|             false|
|    Elizabeth George|     CARELESS IN RED|             false|
|      David Baldacci|     THE WHOLE TRUTH|             false|
|        Troy Denning|          INVINCIBLE|             false|
|          James Frey|BRIGHT SHINY MORNING|             false|
|         Garth Stein|THE ART OF RACING...|            

## [6] Phép toán 'Startswith' — 'Endswith'

In [11]:
dataframe.select("author", "title", dataframe.title.startswith("THE")).show(5)
dataframe.select("author", "title", dataframe.title.endswith("NT")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows

+-----------------+--------------------+-------------------+
|           author|               title|endswith(title, NT)|
+-----------------+--------------------+-------------------+
|    Dean R Koontz|           ODD HOURS|              false|
|  Stephenie Meyer|            THE HOST|              false|
|     Emily Giffin|LOVE THE ONE YOU'...|              false|
|Patricia Cornwell|           THE

## [7] Phép toán 'Substring'

In [12]:
dataframe.select(dataframe.author.substr(1, 3).alias("title")).show(5)
dataframe.select(dataframe.author.substr(3, 6).alias("title")).show(5)
dataframe.select(dataframe.author.substr(1, 6).alias("title")).show(5)

+-----+
|title|
+-----+
|  Dea|
|  Ste|
|  Emi|
|  Pat|
|  Chu|
+-----+
only showing top 5 rows

+------+
| title|
+------+
|an R K|
|epheni|
|ily Gi|
|tricia|
|uck Pa|
+------+
only showing top 5 rows

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



## [8] Thêmcột

In [13]:
from pyspark.sql import functions as F
# Lit() được yêu cầu trong khi chúng tôi đang tạo các cột có giá trị chính xác.
dataframe = dataframe.withColumn('new_column', F.lit('This is a new column'))
display(dataframe)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

## [9] Cập nhậtcột

In [14]:
# Cập nhật cột 'amazon_product_url' bằng 'URL'
dataframe = dataframe.withColumnRenamed('amazon_product_url', 'URL')
dataframe.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, 

## [10] Loại bỏ cột

In [15]:
# 1.
dataframe_remove = dataframe.drop("publisher", "published_date").show(5)
# 2.
dataframe_remove2 = dataframe.drop(dataframe.publisher).drop(dataframe.published_date).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}| {2}|           {1}|            THE HOST|          {3}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|  

## [11] Phép toán 'GroupBy'

In [17]:
# Nhóm theo tác giả, đếm sách của các tác giả trong nhóm
dataframe.groupBy("author").count().show(10)

+--------------------+-----+
|              author|count|
+--------------------+-----+
|          James Frey|    2|
|    Elin Hilderbrand|   58|
|   Sharon Kay Penman|    2|
|         Kate Jacobs|    3|
|       Karen Robards|    6|
|     Gary Shteyngart|    3|
|         Lisa Genova|    7|
|James Patterson a...|   30|
|         Ruth Reichl|    3|
|    Orson Scott Card|    3|
+--------------------+-----+
only showing top 10 rows



## [12] Phép toán 'Filter'

In [18]:
# Lọc các mục tiêu đề
# Chỉ giữ các bản ghi có giá trị 'THE HOST'
dataframe.filter(dataframe["title"] == 'THE HOST').show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|   title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|THE HOST|          {3}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1212192000000}}|Aliens have taken...|{25.99, null}|{{1213488000000}}|Little, Brown| {2}|           {2}|

## [13] Xử lý các giá trị bị thiếu

In [19]:
# Thay thế giá trị null
dataframe.na.fill(4)

DataFrame[_id: struct<$oid:string>, URL: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

In [20]:
# Trả về các hàng giới hạn khung dữ liệu mới có giá trị nulldataframe.na.drop()
dataframe.dropna()

DataFrame[_id: struct<$oid:string>, URL: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

In [21]:
# Trả về khung dữ liệu mới thay thế một giá trị bằng một giá trị khác
dataframe.na.replace(5, 15)

DataFrame[_id: struct<$oid:string>, URL: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

## [14] phân vùng lại

In [22]:
# Dataframe với 10 phân vùng
dataframe.repartition(10).rdd.getNumPartitions()
# Dataframe với 1 phân vùng
dataframe.coalesce(1).rdd.getNumPartitions()

1

## [15] Chạy cáclệnh SQLtrong Spark

In [23]:
# Đăng ký bảng
dataframe.registerTempTable("df")
sc.sql("select * from df").show(3)
sc.sql("""SELECT CASE WHEN description LIKE '%love%' THEN 'Love_Theme'
            WHEN description LIKE '%hate%' THEN 'Hate_Theme' \
            WHEN description LIKE '%happy%' THEN 'Happiness_Theme' \
            WHEN description LIKE '%anger%' THEN 'Anger_Theme' \
            WHEN description LIKE '%horror%' THEN 'Horror_Theme' \
            WHEN description LIKE '%death%' THEN 'Criminal_Theme' \
            WHEN description LIKE '%detective%' THEN 'Mystery_Theme' ELSE 'Other_Themes' END Themes from df").groupBy('Themes').count().show()""")

C:\spark/spark/spark-3.2.3-bin-hadoop2.7\python\pyspark\sql\dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|This is a new column|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{12

ParseException: 
mismatched input '"' expecting {<EOF>, ';'}(line 2, pos 429)

== SQL ==
SELECT CASE WHEN description LIKE '%love%' THEN 'Love_Theme'
            WHEN description LIKE '%hate%' THEN 'Hate_Theme'             WHEN description LIKE '%happy%' THEN 'Happiness_Theme'             WHEN description LIKE '%anger%' THEN 'Anger_Theme'             WHEN description LIKE '%horror%' THEN 'Horror_Theme'             WHEN description LIKE '%death%' THEN 'Criminal_Theme'             WHEN description LIKE '%detective%' THEN 'Mystery_Theme' ELSE 'Other_Themes' END Themes from df").groupBy('Themes').count().show()
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## [16] Xuất dữ liệu

In [24]:
# Chuyển đổi khung dữ liệu thành RDD
rdd_convert = dataframe.rdd
# Chuyển đổi khung dữ liệu thành RDD của chuỗi
dataframe.toJSON().first()
# Lấy nội dung của df dưới dạng Pandas
dataframe.toPandas()

C:\spark/spark/spark-3.2.3-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


Py4JJavaError: An error occurred while calling o126.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 43.0 failed 1 times, most recent failure: Lost task 0.0 in stage 43.0 (TID 44) (MSI executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2454)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2403)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2402)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2402)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(D

## [17] Viết và lưu vào tệp

In [ ]:
# Viết và lưu tệp ở định dạng .parquet
dataframe.select("author", "title", "rank", "description") \
.write \
.save("Rankings_Descriptions.parquet")
# Viết và lưu tệp ở định dạng .json
dataframe.select("author", "title") \
.write \
.save("Authors_Titles.json",format="json")

## [18] Kết thúc phiên Spark

In [ ]:
sc.stop()